In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [3]:
!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.9/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.9/index.html
ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


This did not work, at end of tutorial they included a link to how to do it in colab

In [4]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15819, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 15819 (delta 31), reused 47 (delta 17), pack-reused 15743 (from 1)
Receiving objects: 100% (15819/15819), 6.38 MiB | 6.60 MiB/s, done.
Resolving deltas: 100% (11527/11527), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your en

In [5]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
model_zoo.model_zoo._ModelZooUrls.CONFIG_PATH_TO_URL_SUFFIX.items()

dict_items([('COCO-Detection/faster_rcnn_R_50_C4_1x', '137257644/model_final_721ade.pkl'), ('COCO-Detection/faster_rcnn_R_50_DC5_1x', '137847829/model_final_51d356.pkl'), ('COCO-Detection/faster_rcnn_R_50_FPN_1x', '137257794/model_final_b275ba.pkl'), ('COCO-Detection/faster_rcnn_R_50_C4_3x', '137849393/model_final_f97cb7.pkl'), ('COCO-Detection/faster_rcnn_R_50_DC5_3x', '137849425/model_final_68d202.pkl'), ('COCO-Detection/faster_rcnn_R_50_FPN_3x', '137849458/model_final_280758.pkl'), ('COCO-Detection/faster_rcnn_R_101_C4_3x', '138204752/model_final_298dad.pkl'), ('COCO-Detection/faster_rcnn_R_101_DC5_3x', '138204841/model_final_3e0943.pkl'), ('COCO-Detection/faster_rcnn_R_101_FPN_3x', '137851257/model_final_f6e8b1.pkl'), ('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x', '139173657/model_final_68b088.pkl'), ('COCO-Detection/retinanet_R_50_FPN_1x', '190397773/model_final_bfca0b.pkl'), ('COCO-Detection/retinanet_R_50_FPN_3x', '190397829/model_final_5bd44e.pkl'), ('COCO-Detection/retinane

In [6]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread("desk.jpg")
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(img)

model_final_68b088.pkl: 421MB [00:03, 118MB/s]                            
/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

In [9]:
def detect_objects(img_path):
  img = cv2.imread(img_path)
  outputs = predictor(img)
  v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  return out.get_image()[:, :, ::-1]

In [10]:
import timeit
import pandas as pd
import numpy as np

def benchmark_detect(detect_function, img_path, number=500, repeat=7):
    """
    Benchmarks the detect function by measuring the execution time over a number of iterations.
    Returns a list of execution times in milliseconds.
    """
    times = timeit.repeat(
        stmt='detect_objects("desk.jpg")',
        setup='from __main__ import detect_objects',
        repeat=repeat,  # Number of times the timing loop is repeated
        number=number   # Number of executions per timing loop
    )
    # Convert total time to per-execution time in milliseconds
    per_execution_times = [t / number * 1000 for t in times]
    return per_execution_times

number = 10

repeat = 100  # Number of times the timing loop is repeated

# Run the benchmark
execution_times = benchmark_detect(detect_objects, "desk.jpg", number=number, repeat=repeat)

# Replace 'FrameworkName' with the actual name of the framework
framework_name = 'Detectron2'

df = pd.DataFrame({
    'Framework': [framework_name] * len(execution_times),
    'ExecutionTime': execution_times
})


In [12]:
# Option 1: Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')

csv_path = f'/content/drive/MyDrive/{framework_name}_execution_times.csv'
df.to_csv(csv_path, index=False)
print(f'Data saved to {csv_path}')

# Option 2: Download the CSV file directly
df.to_csv(f'{framework_name}_execution_times.csv', index=False)
from google.colab import files
files.download(f'{framework_name}_execution_times.csv')


Mounted at /content/drive
Data saved to /content/drive/MyDrive/Detectron2_execution_times.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%timeit -n 100 detect_objects("desk.jpg")

609 ms ± 5.54 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
